## Scenario: Loan Approval Prediction (Multilayer Neural Network)
## Context
Banks want to decide whether to approve a loan application. The decision depends on multiple factors, and the relationships are non-linear (not just a simple rule).
Inputs (Features)
Income level (normalized numeric value)
Credit score (normalized numeric value)
#### Output
1 = Loan approved
0 = Loan rejected
Rule (intuitive, hidden from model)
Higher income + good credit score → likely approved.
Low income + poor credit score → likely rejected.
Middle cases depend on combinations (non-linear patterns).

In [8]:

import numpy as np
import pandas as pd

# -----------------------
# 1) Load the dataset
# -----------------------
df = pd.read_csv('loan_approvals_synth.csv')
X = df[['income', 'credit_score']].values.astype(np.float32)
y = df[['approved']].values.astype(np.float32)

# Train/test split (same logic as generation for comparability)
np.random.seed(42)
perm = np.random.permutation(len(X))
train_size = int(0.8 * len(X))
train_idx, test_idx = perm[:train_size], perm[train_size:]
X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

# -----------------------
# 2) MLP architecture
# -----------------------
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def drelu(x):
    return (x > 0).astype(x.dtype)

layers = [2, 16, 16, 8, 1]
L = len(layers) - 1

# He init for ReLU layers; Xavier for last sigmoid
rng = np.random.default_rng(123)
params = {}
for l in range(1, L+1):
    fan_in, fan_out = layers[l-1], layers[l]
    if l < L:
        params[f'W{l}'] = rng.normal(0, np.sqrt(2.0 / fan_in), size=(fan_in, fan_out))
    else:
        params[f'W{l}'] = rng.normal(0, np.sqrt(1.0 / fan_in), size=(fan_in, fan_out))
    params[f'b{l}'] = np.zeros((1, fan_out))

# Adam buffers
beta1, beta2, eps = 0.9, 0.999, 1e-8
opt = {}
for l in range(1, L+1):
    opt[f'mW{l}'] = np.zeros_like(params[f'W{l}'])
    opt[f'vW{l}'] = np.zeros_like(params[f'W{l}'])
    opt[f'mb{l}'] = np.zeros_like(params[f'b{l}'])
    opt[f'vb{l}'] = np.zeros_like(params[f'b{l}'])

def forward(X, params):
    cache = {'A0': X}
    A = X
    # Hidden layers
    for l in range(1, L):
        Z = A @ params[f'W{l}'] + params[f'b{l}']
        A = relu(Z)
        cache[f'Z{l}'] = Z
        cache[f'A{l}'] = A
    # Output
    ZL = A @ params[f'W{L}'] + params[f'b{L}']
    AL = sigmoid(ZL)
    cache[f'Z{L}'] = ZL
    cache[f'A{L}'] = AL
    return AL, cache

def bce(AL, y_true):
    AL = np.clip(AL, 1e-7, 1-1e-7)
    return -np.mean(y_true*np.log(AL) + (1-y_true)*np.log(1-AL))

def backward(cache, params, y_true):
    grads = {}
    m = y_true.shape[0]

    AL = cache[f'A{L}']
    dZL = AL - y_true
    A_prev = cache[f'A{L-1}'] if L-1 > 0 else cache['A0']
    grads[f'dW{L}'] = (A_prev.T @ dZL) / m
    grads[f'db{L}'] = np.sum(dZL, axis=0, keepdims=True) / m

    dA_prev = dZL @ params[f'W{L}'].T
    for l in range(L-1, 0, -1):
        Z = cache[f'Z{l}']
        dZ = dA_prev * drelu(Z)
        A_prev = cache[f'A{l-1}'] if l-1 > 0 else cache['A0']
        grads[f'dW{l}'] = (A_prev.T @ dZ) / m
        grads[f'db{l}'] = np.sum(dZ, axis=0, keepdims=True) / m
        if l > 1:
            dA_prev = dZ @ params[f'W{l}'].T

    return grads

def adam_step(params, grads, opt, lr, t):
    for l in range(1, L+1):
        # weights
        opt[f'mW{l}'] = beta1*opt[f'mW{l}'] + (1-beta1)*grads[f'dW{l}']
        opt[f'vW{l}'] = beta2*opt[f'vW{l}'] + (1-beta2)*(grads[f'dW{l}']**2)
        mW_hat = opt[f'mW{l}'] / (1 - beta1**t)
        vW_hat = opt[f'vW{l}'] / (1 - beta2**t)
        params[f'W{l}'] -= lr * mW_hat / (np.sqrt(vW_hat) + eps)

        # biases
        opt[f'mb{l}'] = beta1*opt[f'mb{l}'] + (1-beta1)*grads[f'db{l}']
        opt[f'vb{l}'] = beta2*opt[f'vb{l}'] + (1-beta2)*(grads[f'db{l}']**2)
        mb_hat = opt[f'mb{l}'] / (1 - beta1**t)
        vb_hat = opt[f'vb{l}'] / (1 - beta2**t)
        params[f'b{l}'] -= lr * mb_hat / (np.sqrt(vb_hat) + eps)

# -----------------------
# 3) Train
# -----------------------
lr = 0.01
epochs = 600
batch_size = 64
n_batches = int(np.ceil(len(X_train) / batch_size))

step = 0
for epoch in range(1, epochs+1):
    idx = np.random.permutation(len(X_train))
    Xs, ys = X_train[idx], y_train[idx]
    for b in range(n_batches):
        s, e = b*batch_size, (b+1)*batch_size
        Xb, yb = Xs[s:e], ys[s:e]
        AL, cache = forward(Xb, params)
        grads = backward(cache, params, yb)
        step += 1
        adam_step(params, grads, opt, lr, step)

    # Optionally print training loss
    if epoch % 100 == 0:
        AL_full, _ = forward(X_train, params)
        print(f'Epoch {epoch:3d}  Loss: {bce(AL_full, y_train):.4f}')

# -----------------------
# 4) Evaluate
# -----------------------
AL_test, _ = forward(X_test, params)
preds = (AL_test >= 0.5).astype(int)
acc = (preds == y_test).mean()
print(f'Test accuracy: {acc*100:.2f}%')


Epoch 100  Loss: 0.3759
Epoch 200  Loss: 0.3680
Epoch 300  Loss: 0.3662
Epoch 400  Loss: 0.3619
Epoch 500  Loss: 0.3602
Epoch 600  Loss: 0.3541
Test accuracy: 77.92%


MUKTI_LAYER_NEURAL_ NETWORK


In [18]:

import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

df = pd.read_csv('loan_approvals_synth.csv')
X = df[['income', 'credit_score']].values
y = df['approved'].values

# Train/test
import numpy as np
np.random.seed(42)
perm = np.random.permutation(len(X))
train_size = int(0.8*len(X))
X_train, y_train = X[perm[:train_size]], y[perm[:train_size]]
X_test,  y_test  = X[perm[train_size:]], y[perm[train_size:]]

model = keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=60, batch_size=64, verbose=1)
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {acc*100:.2f}%')


Epoch 1/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5352 - loss: 0.6852 - val_accuracy: 0.6615 - val_loss: 0.6405
Epoch 2/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7292 - loss: 0.6040 - val_accuracy: 0.7448 - val_loss: 0.5403
Epoch 3/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8112 - loss: 0.5373 - val_accuracy: 0.7865 - val_loss: 0.4883
Epoch 4/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8281 - loss: 0.5059 - val_accuracy: 0.8229 - val_loss: 0.4651
Epoch 5/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8281 - loss: 0.4872 - val_accuracy: 0.8125 - val_loss: 0.4566
Epoch 6/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8294 - loss: 0.4748 - val_accuracy: 0.8177 - val_loss: 0.4459
Epoch 7/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8307 - loss: 0.4640 - val_accuracy: 0.8021 - val_loss: 0.4503
Epoch 8/60
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8346 - loss: 0.4536 - val_accuracy: 0.8073 - va